In [1]:
import numpy as np
import random
import json

with open("problem_data.json", 'r') as file:
    problem_data = json.load(file)

class Question:
    def __init__(self, question_id, title, difficulty, accuracy):
        self.question_id = question_id
        self.title = title
        self.difficulty = self.map_difficulty(difficulty)
        self.accuracy = accuracy

    @staticmethod
    def map_difficulty(difficulty):
        """Convert string difficulty to numeric value."""
        if difficulty == "easy":
            return 1
        elif difficulty == "medium":
            return 2
        elif difficulty == "hard":
            return 3
        return 0

class User:
    def __init__(self, user_id, initial_skill=0):
        self.user_id = user_id
        self.skill = initial_skill  # Initialize user skill level
        self.history = []  # To store the user’s performance on each question
        self.attempted_questions = set()  # To track attempted questions
        self.streak = 0  # Track consecutive correct answers for extra skill boost

class RecommenderModel:
    def __init__(self, questions, learning_rate=0.2):
        self.questions = questions
        self.learning_rate = learning_rate  # Adjust skill update rate to be more impactful

    def probability_of_solving(self, skill, difficulty):
        """Calculate the probability that a user with a given skill level solves a question of a given difficulty."""
        return 1 / (1 + np.exp(-(skill - difficulty)))

    def update_skill(self, user, difficulty, solved):
        """Update skill based on question outcome (solved or not solved)."""
        prob_solved = self.probability_of_solving(user.skill, difficulty)
        if solved:
            # Increase skill more significantly for correct answers and add streak bonus
            skill_increase = self.learning_rate * (1 - prob_solved) + (user.streak * 0.05)
            user.skill += skill_increase
            user.streak += 1  # Increment streak for consecutive successes
        else:
            # Only a small decrease for incorrect answers, reset streak
            user.skill -= self.learning_rate * prob_solved
            user.streak = 0

        # Ensure skill stays within reasonable bounds
        user.skill = max(user.skill, -0.2)
        return user.skill

    def recommend_next_question(self, user):
        """Recommend the next question based on user's current skill level."""
        # Filter out already attempted questions
        suitable_questions = [
            q for q in self.questions if q.question_id not in user.attempted_questions
        ]

        if not suitable_questions:  # If all questions have been attempted
            return None

        # Sort by the absolute difference in difficulty from user's skill
        suitable_questions = sorted(suitable_questions, key=lambda q: abs(q.difficulty - user.skill))

        # Set a difficulty tolerance to allow faster progression to medium/hard questions
        tolerance = 1.5 if user.skill >= 1 else 1.0
        close_questions = [q for q in suitable_questions if abs(q.difficulty - user.skill) <= tolerance]

        # If we have close enough questions, select randomly from them
        if close_questions:
            next_question = random.choice(close_questions)
        else:
            # Otherwise, pick the closest question available
            next_question = suitable_questions[0]

        return next_question

    def attempt_question(self, user, question):
        """Simulate the user attempting a question with manual input."""
        print(f"\nDid you solve the question '{question.title}'? (yes/no): ", end="")
        user_input = input().strip().lower()
        solved = user_input == "yes"

        # Update user’s skill based on their input
        user.skill = self.update_skill(user, question.difficulty, solved)

        # Log the attempt in user's history
        user.history.append({
            'question_id': question.question_id,
            'title': question.title,
            'difficulty': question.difficulty,
            'solved': solved,
            'updated_skill': user.skill
        })

        # Add the question to the user's attempted questions
        user.attempted_questions.add(question.question_id)

        return solved

# Initialize questions from the problem_data
questions = [
    Question(
        question_id=q_id,
        title=info["title"],
        difficulty=info["difficulty"],
        accuracy=info["accuracy"]
    )
    for q_id, info in problem_data.items()
]

# Initialize a user
user = User(user_id=1)

# Create the recommender model
recommender = RecommenderModel(questions)

# Simulate a series of attempts
for i in range(10):  # Simulate 10 question attempts
    # Recommend the next question based on user's skill
    next_question = recommender.recommend_next_question(user)
    
    if next_question is None:  # If all questions have been attempted
        print("All questions have been attempted.")
        break

    print(f"Attempt {i + 1}:")
    print(f"Recommended Question ID: {next_question.question_id}, Title: {next_question.title}, Difficulty: {next_question.difficulty}")

    # User attempts the question with manual input
    solved = recommender.attempt_question(user, next_question)
    print(f"User Skill: {user.skill:.2f}, Solved: {'Yes' if solved else 'No'}\n")

# Display user history
print("User Attempt History:")
for attempt in user.history:
    print(attempt)


Attempt 1:
Recommended Question ID: 2880, Title: Select Data, Difficulty: 1

Did you solve the question 'Select Data'? (yes/no): User Skill: 0.15, Solved: Yes

Attempt 2:
Recommended Question ID: 2833, Title: Furthest Point From Origin, Difficulty: 1

Did you solve the question 'Furthest Point From Origin'? (yes/no): User Skill: 0.34, Solved: Yes

Attempt 3:
Recommended Question ID: 1961, Title: Check If String Is a Prefix of Array, Difficulty: 1

Did you solve the question 'Check If String Is a Prefix of Array'? (yes/no): User Skill: 0.27, Solved: No

Attempt 4:
Recommended Question ID: 1378, Title: Replace Employee ID With The Unique Identifier, Difficulty: 1

Did you solve the question 'Replace Employee ID With The Unique Identifier'? (yes/no): User Skill: 0.40, Solved: Yes

Attempt 5:
Recommended Question ID: 2798, Title: Number of Employees Who Met the Target, Difficulty: 1

Did you solve the question 'Number of Employees Who Met the Target'? (yes/no): User Skill: 0.58, Solved: Ye

#### **With Time and Likingness**

In [3]:
import numpy as np
import random
import json
import time

# Load problem data from a JSON file
with open("problem_data.json", 'r') as file:
    problem_data = json.load(file)

class Question:
    def __init__(self, question_id, title, difficulty, accuracy, topics):
        self.question_id = question_id
        self.title = title
        self.difficulty = self.map_difficulty(difficulty)
        self.accuracy = accuracy
        self.topics = topics  # Topics as a list of strings

    @staticmethod
    def map_difficulty(difficulty):
        """Convert string difficulty to numeric value."""
        if difficulty == "easy":
            return 1
        elif difficulty == "medium":
            return 2
        elif difficulty == "hard":
            return 3
        return 0

class User:
    def __init__(self, user_id, initial_skill=0):
        self.user_id = user_id
        self.skill = initial_skill
        self.history = []
        self.attempted_questions = set()
        self.streak = 0
        self.liked_topics = set()  # Track liked question topics

class RecommenderModel:
    def __init__(self, questions, learning_rate=0.2, base_time=60):
        self.questions = questions
        self.learning_rate = learning_rate
        self.base_time = base_time  # Base time (in seconds) for ideal skill updates

    def probability_of_solving(self, skill, difficulty):
        return 1 / (1 + np.exp(-(skill - difficulty)))

    def update_skill(self, user, difficulty, solved, time_taken):
        prob_solved = self.probability_of_solving(user.skill, difficulty)
        
        # Adjust skill based on whether question was solved and time taken
        if solved:
            # Calculate time factor to scale skill adjustment based on time taken
            time_factor = max(0.5, min(2.0, self.base_time / (time_taken + 1)))  # Normalize time impact
            skill_increase = self.learning_rate * (1 - prob_solved) * time_factor + (user.streak * 0.05)
            user.skill += skill_increase
            user.streak += 1
        else:
            user.skill -= self.learning_rate * prob_solved
            user.streak = 0

        user.skill = max(user.skill, -0.2)
        return user.skill

    def recommend_next_question(self, user):
        suitable_questions = [
            q for q in self.questions if q.question_id not in user.attempted_questions
        ]

        if not suitable_questions:
            return None

        # Filter for questions that match user's liked topics, if any are liked
        if user.liked_topics:
            liked_topic_questions = [
                q for q in suitable_questions if set(q.topics).intersection(user.liked_topics)
            ]
            if liked_topic_questions:
                suitable_questions = liked_topic_questions

        # Sort by the absolute difference in difficulty from user's skill
        suitable_questions = sorted(suitable_questions, key=lambda q: abs(q.difficulty - user.skill))

        tolerance = 1.5 if user.skill >= 1 else 1.0
        close_questions = [q for q in suitable_questions if abs(q.difficulty - user.skill) <= tolerance]

        next_question = random.choice(close_questions) if close_questions else suitable_questions[0]

        return next_question

    def attempt_question(self, user, question):
        print(f"\nAttempting question '{question.title}' in topics {question.topics}...")
        
        # Start timing the attempt
        start_time = time.time()
        
        solved_input = input("Did you solve the question? (yes/no): ").strip().lower()
        solved = solved_input == "yes"
        
        # Record end time and calculate time taken
        time_taken = time.time() - start_time

        # Update skill with time taken into account
        user.skill = self.update_skill(user, question.difficulty, solved, time_taken)

        # Ask if user liked the question for preference tracking
        liked_input = input("Did you like this question? (yes/no): ").strip().lower()
        if liked_input == "yes":
            user.liked_topics.update(question.topics)  # Add all topics to liked topics

        # Log attempt
        user.history.append({
            'question_id': question.question_id,
            'title': question.title,
            'difficulty': question.difficulty,
            'topics': question.topics,
            'solved': solved,
            'time_taken': time_taken,
            'liked': liked_input == "yes",
            'updated_skill': user.skill
        })

        user.attempted_questions.add(question.question_id)

        return solved

# Initialize questions from the problem_data
questions = [
    Question(
        question_id=q_id,
        title=info["title"],
        difficulty=info["difficulty"],
        accuracy=info["accuracy"],
        topics=info["topics"]
    )
    for q_id, info in problem_data.items()
]

# Initialize a user
user = User(user_id=1)

# Create the recommender model
recommender = RecommenderModel(questions)

# Simulate a series of attempts
for i in range(10):  # Simulate 10 question attempts
    next_question = recommender.recommend_next_question(user)
    
    if next_question is None:
        print("All questions have been attempted.")
        break

    print(f"\nAttempt {i + 1}:")
    print(f"Recommended Question ID: {next_question.question_id}, Title: {next_question.title}, Difficulty: {next_question.difficulty}")

    # User attempts the question with manual input
    solved = recommender.attempt_question(user, next_question)
    print(f"User Skill: {user.skill:.2f}, Solved: {'Yes' if solved else 'No'}\n")

# Display user history
print("User Attempt History:")
for attempt in user.history:
    print(attempt)



Attempt 1:
Recommended Question ID: 1890, Title: The Latest Login in 2020, Difficulty: 1

Attempting question 'The Latest Login in 2020' in topics ['database']...
User Skill: -0.05, Solved: No


Attempt 2:
Recommended Question ID: 1, Title: Two Sum, Difficulty: 1

Attempting question 'Two Sum' in topics ['array', 'hash-table']...
User Skill: 0.24, Solved: Yes


Attempt 3:
Recommended Question ID: 682, Title: Baseball Game, Difficulty: 1

Attempting question 'Baseball Game' in topics ['array', 'stack', 'simulation']...
User Skill: 0.57, Solved: Yes


Attempt 4:
Recommended Question ID: 448, Title: Find All Numbers Disappeared in an Array, Difficulty: 1

Attempting question 'Find All Numbers Disappeared in an Array' in topics ['array', 'hash-table']...
User Skill: 0.91, Solved: Yes


Attempt 5:
Recommended Question ID: 485, Title: Max Consecutive Ones, Difficulty: 1

Attempting question 'Max Consecutive Ones' in topics ['array']...
User Skill: 1.27, Solved: Yes


Attempt 6:
Recommended 